## Siniestro Viales

Importando librerias 

In [223]:

import numpy as np 
import pandas as pd 
#import pandas_alive
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import geojson
import folium
from folium.plugins import HeatMap
from summarytools import dfSummary


 ##ETL Extracción, transformación y carga

In [147]:
homicidios_hechos = pd.read_excel("homicidios.xlsx", sheet_name="HECHOS")

In [148]:
homicidios_victimas = pd.read_excel("lesiones.xlsx", sheet_name="VICTIMAS")

In [150]:
comunas= pd.read_excel("comunas.xlsx")

Tratamiento de los datos del dataset homicidios - hechos

In [151]:
homicidios_hechos

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,NaN,MOM,RIESTRA AV. y MOM,7,Point (102728.60090138 98186.24929177),-58.43353773,-34.64561636,MOTO-AUTO,MOTO,AUTO
692,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",9,Point (99624.29795829 97569.69801131),-58.46739825,-34.65117757,MOTO-AUTO,MOTO,AUTO
693,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,NaN,TERRADA,GAONA AV. y TERRADA,11,Point (99116.45492358 101045.23284826),-58.47293407,-34.61984745,MOTO-CARGAS,MOTO,CARGAS
694,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,4071.0,NaN,"PERON, EVA AV. 4071",9,Point (99324.54463985 97676.26932409),-58.47066794,-34.65021673,AUTO-CARGAS,AUTO,CARGAS


In [152]:
homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   696 non-null    object        
 7   HH                     696 non-null    object        
 8   LUGAR_DEL_HECHO        696 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Examen de nulos y duplicados

In [153]:
homicidios_hechos.isnull().sum()

ID                         0
N_VICTIMAS                 0
FECHA                      0
AAAA                       0
MM                         0
DD                         0
HORA                       0
HH                         0
LUGAR_DEL_HECHO            0
TIPO_DE_CALLE              0
Calle                      1
Altura                   567
Cruce                    171
Dirección Normalizada      8
COMUNA                     0
XY (CABA)                  0
pos x                      0
pos y                      0
PARTICIPANTES              0
VICTIMA                    0
ACUSADO                    0
dtype: int64

Se analizan las columnas con datos nulos

In [154]:
homicidios_hechos[["Altura", "Calle", "Dirección Normalizada", "Cruce"]][:5]

,Altura,Calle,Dirección Normalizada,Cruce
0,NaN,PIEDRA BUENA AV.,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...","FERNANDEZ DE LA CRUZ, F., GRAL. AV."
1,NaN,"PAZ, GRAL. AV.","PAZ, GRAL. AV. y DE LOS CORRALES AV.",DE LOS CORRALES AV.
2,2034.0,ENTRE RIOS AV.,ENTRE RIOS AV. 2034,NaN
3,NaN,LARRAZABAL AV.,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.","VILLEGAS, CONRADO, GRAL."
4,NaN,SAN JUAN AV.,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.","SAENZ PE?A, LUIS, PRES."


In [155]:
homicidios_hechos[homicidios_hechos["Dirección Normalizada"].isnull()][["Dirección Normalizada", "Cruce","Altura", "Calle"]]

,Dirección Normalizada,Cruce,Altura,Calle
38,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
106,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA
119,NaN,NaN,NaN,NaN
180,NaN,NaN,NaN,AUTOPISTA PERITO MORENO
181,NaN,NaN,NaN,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.
313,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
546,NaN,NaN,NaN,"LUGONES, LEOPOLDO AV."
621,NaN,NaN,NaN,AUTOPISTA BUENOS AIRES - LA PLATA


Analizando los datos de la Altura se procede a eliminarla debido a la gran cantidad de faltantes y no existiendo manera de poder rellenar las colummas; ya que altura se refiere a la numeración de la calle donde se produce el siniestro y la mayoria de los casos se producen en cruces o esquinas. La columna Cruce solo va a tener dato cuando el hecho se porduce en un cruce de calles, por lo que se conserva de esta manera. La columna Dirección Normalizada no tiene posibilidad de completarse con las columnas del dataset; pero contiene solo 1,1% de nulos por lo se conserva de esta manera.

In [156]:
# Se elimina la columna "Altura"
homicidios_hechos= homicidios_hechos.drop("Altura", axis=1)
homicidios_hechos.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [157]:
homicidios_hechos.duplicated().sum()

0

Para el caso no se encontraron datos duplicado

Se regularizan los tipos de datos de las columnas

In [158]:
homicidios_hechos["HH"].unique()

array([4, 1, 7, 0, 5, 18, 19, 15, 11, 22, 16, 9, 23, 6, 10, 17, 12, 8, 20,
       21, 14, 3, 2, 13, 'SD'], dtype=object)

El campo que contiene SD, no se modifica debido a que en la columna HORA donde tambien debería existir dato, se encuentra SD. Por lo que no podemos calcular un reemplazo. Otra opción sería calcular la mediana de las horas y reemplazarlas pero podria no ser representativo en este caso. Por lo que se define mantenerlo en SD

In [159]:
# Cambio el dato horas  a tipo de dato numerico y luego a entero 
homicidios_hechos["HH"] = pd.to_numeric(homicidios_hechos ["HH"], errors="coerce")
homicidios_hechos["HH"] = homicidios_hechos["HH"].astype("Int64")

In [160]:
# Filtrar filas con posiciones no válidas
valid_positions = ~(homicidios_hechos['pos x'].str.contains('[^0-9.,-]') | homicidios_hechos['pos y'].str.contains('[^0-9.,-]') | (homicidios_hechos['pos x'] == '.') | (homicidios_hechos['pos y'] == '.'))

# Establecer posiciones no válidas en NaN
homicidios_hechos.loc[~valid_positions, ['pos x', 'pos y']] = np.nan

# Convertir las cadenas a tipo numérico (float)
homicidios_hechos['pos x'] = homicidios_hechos['pos x'].str.replace(',', '.').astype(float)
homicidios_hechos['pos y'] = homicidios_hechos['pos y'].str.replace(',', '.').astype(float)

In [161]:
homicidios_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO


La columna XY (CABA) posee coordenadas para poder ubicar en un mapa , las extraemos para poder utilizar

In [162]:
def extraer_coordenadas(texto):
    # Utilizamos una expresión regular para extraer las coordenadas
    coordenadas = re.findall(r'\d+\.\d+', texto)
    if len(coordenadas) == 2:
        return float(coordenadas[0]), float(coordenadas[1])
    else:
        return None, None

# Aplicar la función a la columna 'XY (CABA)'
homicidios_hechos['coordenada_x'], homicidios_hechos['coordenada_y'] = zip(*homicidios_hechos['XY (CABA)'].map(extraer_coordenadas))

# Filtrar los registros que no tienen coordenadas válidas
homicidios_hechos = homicidios_hechos[(homicidios_hechos['coordenada_x'].notnull()) & (homicidios_hechos['coordenada_y'].notnull())]
homicidios_hechos.head(3)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO,coordenada_x,coordenada_y
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO-AUTO,MOTO,AUTO,98896.782384,93532.434378
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO-PASAJEROS,AUTO,PASAJEROS,95832.055711,95505.416420
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.390403,-34.631894,MOTO-AUTO,MOTO,AUTO,106684.290900,99706.576878


In [228]:
#Se observan las columnas resultantes
homicidios_hechos.columns


Index(['Id', 'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa',
       'Hora completa', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Coordenada x', 'Coordenada y',
       'Id y', 'Objeto', 'Barrios', 'Perimetro', 'Area', 'Día semana'],
      dtype='object')

Para agregar valor al dataset, agregaremos la descripción de cada una de las comunas, con los Barrios que la conforman.

Para eso utilizaremos el dataset 'comunas' importado junto al dataset homicidios. Este dataset fue descargado del siguiente sitio web: https://buenosaires.gob.ar/comunas

In [237]:
comunas.columns


Index(['ID', 'OBJETO', 'COMUNA', 'BARRIOS', 'PERIMETRO', 'AREA'], dtype='object')

In [236]:
homicidios_hechos.head(5)

,Id,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora completa,Lugar del hecho,Tipo de calle,...,Victima,Acusado,Coordenada x,Coordenada y,Id y,Objeto,Barrios,Perimetro,Area,Día semana
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,MOTO,AUTO,98896.782384,93532.434378,12,LIMITE COMUNAL,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI,20935.012513,2.221690e+07,Viernes
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,AUTO,PASAJEROS,95832.055711,95505.416420,5,LIMITE COMUNAL,LINIERS - MATADEROS - PARQUE AVELLANEDA,21411.738344,1.650531e+07,Sabado
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,MOTO,AUTO,106684.290900,99706.576878,14,LIMITE COMUNAL,CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...,36102.201573,1.779457e+07,Domingo
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,MOTO,SD,99840.652248,94269.165344,12,LIMITE COMUNAL,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI,20935.012513,2.221690e+07,Domingo
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,MOTO,PASAJEROS,106980.328279,100752.169158,14,LIMITE COMUNAL,CONSTITUCION - MONTSERRAT - PUERTO MADERO - RE...,36102.201573,1.779457e+07,Jueves


Estenderizamos las columnas como buenas practica 

In [239]:
#Cambio la primer letra a mayúscula
homicidios_hechos.columns = [x.capitalize() for x in homicidios_hechos.columns]
# Reemplazo guiones por espacios
homicidios_hechos.columns = homicidios_hechos.columns.str.replace("_", " ")
#Renombra columnas 
homicidios_hechos= homicidios_hechos.rename(columns={"N victimas": "Num víctimas",
                                             'Id x':"Id",        
                                            "Aaaa": "Año",
                                            "Mm":"Mes",
                                            "Dd": "Día",
                                            "Hora": "Hora completa",
                                            "Hh": "Hora"}) 

homicidios_hechos.columns

Index(['Id', 'Num víctimas', 'Fecha', 'Año', 'Mes', 'Día', 'Hora completa',
       'Hora completa', 'Lugar del hecho', 'Tipo de calle', 'Calle', 'Cruce',
       'Dirección normalizada', 'Comuna', 'Xy (caba)', 'Pos x', 'Pos y',
       'Participantes', 'Victima', 'Acusado', 'Coordenada x', 'Coordenada y',
       'Id y', 'Objeto', 'Barrios', 'Perimetro', 'Area', 'Día semana'],
      dtype='object')

Cuando analizamos el df notamos los dias de semana se identidican por id numerico y no por nombre. Procedemos a modificar la columna para una mejor lectura de saber que dia de la semana ocurren los siniestros 

In [240]:
homicidios_hechos.Día

0       1
1       2
2       3
3      10
4      21
       ..
677    13
678    20
679    30
680    15
681    18
Name: Día, Length: 682, dtype: int64

In [241]:
#Creo una columna nueva `Día semana` que contenga el nombre del día de la semana
homicidios_hechos['Día semana'] = homicidios_hechos['Fecha'].dt.strftime('%A')

#Creo un diccionario para modificar a español los nombres de los dás de la semana y lo aplico a la columna nueva
dic={  'Friday':'Viernes',
       'Saturday': 'Sabado',
       'Sunday': 'Domingo',  
       'Thursday': 'Jueves',
        'Monday': 'Lunes',
        'Wednesday': 'Miercoles',
       'Tuesday': 'Martes'}
homicidios_hechos['Día semana'] =homicidios_hechos['Día semana'].replace(dic)

In [242]:
homicidios_hechos.head(2)

,Id,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora completa,Lugar del hecho,Tipo de calle,...,Victima,Acusado,Coordenada x,Coordenada y,Id y,Objeto,Barrios,Perimetro,Area,Día semana
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,MOTO,AUTO,98896.782384,93532.434378,12,LIMITE COMUNAL,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI,20935.012513,2.221690e+07,Viernes
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,AUTO,PASAJEROS,95832.055711,95505.416420,5,LIMITE COMUNAL,LINIERS - MATADEROS - PARQUE AVELLANEDA,21411.738344,1.650531e+07,Sabado


Hacemos un analisis descriptivo de los analisi de los datos 

In [243]:
dfSummary(homicidios_hechos)

AttributeError: 'DataFrame' object has no attribute 'dtype'

Transformación de los datos

In [190]:
display(homicidios_victimas.sample(3))

,Id,Año,Mm,Dd,Fecha,Víctima,Sexo,Edad victima,Gravedad
25827,LC-2021-0240084,2021,5,28,2021-05-28,SD,SD,SD,SD
5623,LC-2019-0359019,2019,6,8,2019-06-08,sd,Mujer,SD,SD
12688,LC-2020-0042151,2020,1,20,2020-01-20,CICLISTA,Varon,25,sd


In [191]:
print(homicidios_victimas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Id            27605 non-null  object        
 1   Año           27605 non-null  int64         
 2   Mm            27605 non-null  int64         
 3   Dd            27605 non-null  int64         
 4   Fecha         27605 non-null  datetime64[ns]
 5   Víctima       27605 non-null  object        
 6   Sexo          27605 non-null  object        
 7   Edad victima  27605 non-null  object        
 8   Gravedad      27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB
None


In [192]:
print(homicidios_victimas.columns)

Index(['Id', 'Año', 'Mm', 'Dd', 'Fecha ', 'Víctima', 'Sexo', 'Edad victima',
       'Gravedad'],
      dtype='object')


In [197]:
#Cambio la columna edad a numerico y luego a tipo entero
homicidios_victimas ["Edad victima"] = pd.to_numeric(homicidios_victimas["Edad victima"], errors="coerce")
homicidios_victimas ["Edad victima"] = homicidios_victimas["Edad victima"].astype("Int64")

In [198]:
homicidios_victimas["Edad victima"] = homicidios_victimas["Edad victima"].fillna(0.0)

In [199]:
#Controlo los datos de la columna
homicidios_victimas ["Edad victima"].unique()

<IntegerArray>
[ 57,   0,  45,  27,  32,  21,  37,  33,  25,  23,  42,  53,   1,  62,  67,
  75,  40,  48,   3,  49,  34,  52,  44,  28,  59,  50,  73,  26,  51,  16,
  43,  38,  64,  36,  84,  22,  20,   5,  39,  56,  69,  55,  46,  29,  41,
  54,  74,  70,   6,  30,  12,  83,  24,  35,  68,  15,  31,  19,   9,   2,
  13,  18,  17,  58,  60,  47,   7,  63,  66,  80,  65,  71,  79,  85,  61,
  77,  90,  76,  89,  78,  91,  97,  10,  72,  88,  11,  81,   8,  14,  96,
  82,  87,  86,   4,  92,  94, 100,  93,  95]
Length: 99, dtype: Int64

In [213]:
homicidios_victimas.head(10)

,Id,Año,Mm,Dd,Fecha,Víctima,Sexo,Edad victima,Gravedad,Rango etario
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD,Entre 41-60
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,0,SD,Menos de 20
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,0,SD,Menos de 20
3,LC-2019-0000082,2019,1,1,2019-01-01,sd,Varon,45,SD,Entre 41-60
4,LC-2019-0000082,2019,1,1,2019-01-01,sd,Mujer,45,SD,Entre 41-60
5,LC-2019-0000082,2019,1,1,2019-01-01,sd,Varon,27,SD,Entre 21- 40
6,LC-2019-0000082,2019,1,1,2019-01-01,sd,Mujer,32,SD,Entre 21- 40
7,LC-2019-0000136,2019,1,1,2019-01-01,sd,Varon,21,SD,Entre 21- 40
8,LC-2019-0000136,2019,1,1,2019-01-01,sd,Mujer,21,SD,Entre 21- 40
9,LC-2019-0000142,2019,1,1,2019-01-01,sd,Varon,37,SD,Entre 21- 40


In [201]:
homicidios_victimas.isnull().sum()

Id              0
Año             0
Mm              0
Dd              0
Fecha           0
Víctima         0
Sexo            0
Edad victima    0
Gravedad        0
dtype: int64

Observamos que no se encuentran datos vacios ni nulos 

In [214]:
# Se utiliza la función `pd.cut()`, que permite agrupar valores numéricos en intervalos o categorías


# Definir los límites de los rangos etarios y las etiquetas
bins = [0, 20, 40, 60, 80, 100]
labels = ['Menos de 20', ' Entre 21- 40', 'Entre 41-60', 'Entre 61-80', 'Más de 80']

# Utilizar pd.cut() para crear la columna de rango etario
homicidios_victimas['Rango_etario'] = pd.cut(homicidios_victimas['Edad victima'], bins=bins, labels=labels, right=False)

homicidios_victimas.head(3)

,Id,Año,Mm,Dd,Fecha,Víctima,Sexo,Edad victima,Gravedad,Rango etario,Rango_etario
0,LC-2019-0000053,2019,1,1,2019-01-01,sd,Varon,57,SD,Entre 41-60,Entre 41-60
1,LC-2019-0000063,2019,1,1,2019-01-01,sd,SD,0,SD,Menos de 20,Menos de 20
2,LC-2019-0000079,2019,1,1,2019-01-01,sd,Varon,0,SD,Menos de 20,Menos de 20


A continuación se estandarizan las mayúsculas y minúsculas del dataset

In [215]:
# Se coloca la primera en mayúscula
homicidios_victimas.columns = [x.capitalize() for x in homicidios_victimas.columns]
# Se reemplazan los guiones por espacios
homicidios_victimas.columns = homicidios_victimas.columns.str.replace('_', ' ')
# Se renombran algunas columnas
homicidios_victimas = homicidios_victimas.rename(columns={'Id hecho':'Id',
                                                      'Aaa':'Año',
                                                      'Mes':'Mes',
                                                      'Día':'Día',
                                                      'Vehiculo victima':'Víctima'})
homicidios_victimas.columns

Index(['Id', 'Año', 'Mm', 'Dd', 'Fecha ', 'Víctima', 'Sexo', 'Edad victima',
       'Gravedad', 'Rango etario', 'Rango etario'],
      dtype='object')

In [216]:
#Se busca un registro para comparar con el mismo registro en el otro dataset
homicidios_hechos[homicidios_hechos['Id']=="2019-0012"]

,Id,Num víctimas,Fecha,Año,Mes,Día,Hora completa,Hora completa,Lugar del hecho,Tipo de calle,...,Victima,Acusado,Coordenada x,Coordenada y,Id y,Objeto,Barrios,Perimetro,Area,Día semana
418,2019-0012,1,2019-02-17,2019,2,17,03:16:00,3,"CASTAÑARES AV. Y GORDILLO, TIMOTEO",AVENIDA,...,MOTO,AUTO,98072.989594,94742.808048,12,LIMITE COMUNAL,VILLA LUGANO - VILLA RIACHUELO - VILLA SOLDATI,20935.012513,2.221690e+07,Domingo


In [219]:
homicidios_victimas[homicidios_victimas ['Id']=="2019-0012"]

,Id,Año,Mm,Dd,Fecha,Víctima,Sexo,Edad victima,Gravedad,Rango etario,Rango etario


In [221]:
homicidios_victimas.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Id            27605 non-null  object        
 1   Año           27605 non-null  int64         
 2   Mm            27605 non-null  int64         
 3   Dd            27605 non-null  int64         
 4   Fecha         27605 non-null  datetime64[ns]
 5   Víctima       27605 non-null  object        
 6   Sexo          27605 non-null  object        
 7   Edad victima  27605 non-null  Int64         
 8   Gravedad      27605 non-null  object        
 9   Rango etario  27604 non-null  category      
 10  Rango etario  27604 non-null  category      
dtypes: Int64(1), category(2), datetime64[ns](1), int64(3), object(4)
memory usage: 2.0+ MB


In [207]:
print(homicidios_victimas.columns)

Index(['Id', 'Año', 'Mm', 'Dd', 'Fecha ', 'Víctima', 'Sexo', 'Edad victima',
       'Gravedad', 'Rango etario'],
      dtype='object')


In [208]:
homicidios_victimas.head

<bound method NDFrame.head of                     Id   Año  Mm  Dd     Fecha              Víctima   Sexo  \
0      LC-2019-0000053  2019   1   1 2019-01-01                  sd  Varon   
1      LC-2019-0000063  2019   1   1 2019-01-01                  sd     SD   
2      LC-2019-0000079  2019   1   1 2019-01-01                  sd  Varon   
3      LC-2019-0000082  2019   1   1 2019-01-01                  sd  Varon   
4      LC-2019-0000082  2019   1   1 2019-01-01                  sd  Mujer   
...                ...   ...  ..  ..        ...                 ...    ...   
27600  LC-2021-0451911  2021   9  11 2021-09-11  TRANSPORTE PUBLICO  Varon   
27601  LC-2021-0530228  2021  10  25 2021-10-25  TRANSPORTE PUBLICO  Mujer   
27602  LC-2021-0530228  2021  10  25 2021-10-25  TRANSPORTE PUBLICO  Mujer   
27603  LC-2021-0201378  2021   5   2 2021-05-02                MOTO  Varon   
27604  LC-2021-0229350  2021   5  19 2021-05-19                MOTO  Varon   

       Edad victima Gravedad   Ra

Unificación de los dataset

In [209]:
siniestros= homicidios_victimas.merge(homicidios_hechos, on="Id", how="left")
siniestros.columns

Index(['Id', 'Año_x', 'Mm', 'Dd', 'Fecha ', 'Víctima', 'Sexo', 'Edad victima',
       'Gravedad', 'Rango etario', 'Num víctimas', 'Fecha', 'Año_y', 'Mes',
       'Día', 'Hora completa', 'Hora completa', 'Lugar del hecho',
       'Tipo de calle', 'Calle', 'Cruce', 'Dirección normalizada', 'Comuna',
       'Xy (caba)', 'Pos x', 'Pos y', 'Participantes', 'Victima', 'Acusado',
       'Coordenada x', 'Coordenada y', 'Id y', 'Objeto', 'Barrios',
       'Perimetro', 'Area', 'Día semana'],
      dtype='object')

In [210]:
siniestros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Id                     27605 non-null  object        
 1   Año_x                  27605 non-null  int64         
 2   Mm                     27605 non-null  int64         
 3   Dd                     27605 non-null  int64         
 4   Fecha                  27605 non-null  datetime64[ns]
 5   Víctima                27605 non-null  object        
 6   Sexo                   27605 non-null  object        
 7   Edad victima           27605 non-null  Int64         
 8   Gravedad               27605 non-null  object        
 9   Rango etario           27604 non-null  category      
 10  Num víctimas           0 non-null      float64       
 11  Fecha                  0 non-null      datetime64[ns]
 12  Año_y                  0 non-null      float64       
 13  M

In [211]:
siniestros.describe()

,Año_x,Mm,Dd,Fecha,Edad victima,Num víctimas,Fecha,Año_y,Mes,Día,Hora completa,Comuna,Pos x,Pos y,Coordenada x,Coordenada y,Id y,Perimetro,Area
count,27605.000000,27605.000000,27605.000000,27605,27605.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,2019.897265,6.618113,15.620721,2020-05-27 17:41:20.289802752,31.616809,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2019.000000,1.000000,1.000000,2019-01-01 00:00:00,0.0,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2019.000000,3.000000,8.000000,2019-07-13 00:00:00,22.0,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2020.000000,7.000000,16.000000,2020-03-17 00:00:00,31.0,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2021.000000,10.000000,23.000000,2021-04-05 00:00:00,43.0,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2021.000000,12.000000,31.000000,2021-12-31 00:00:00,100.0,NaN,NaT,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.860899,3.479295,8.684067,NaN,19.678659,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Al hacer el merge vemos que hay datos que no son enteros se hace la transformacion 

In [121]:
siniestros.head(3)

,Id,Sexo,Edad victima,Gravedad,Rango etario,Num víctimas,Fecha,Año,Mes,Día,...,Victima,Acusado,Coordenada x,Coordenada y,Id y,Objeto,Barrios,Perimetro,Area,Día semana
0,LC-2019-0000053,Varon,57,SD,Entre 41-60,0,NaT,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LC-2019-0000063,SD,0,SD,Menos de 20,0,NaT,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LC-2019-0000079,Varon,0,SD,Menos de 20,0,NaT,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
